In [ ]:
import numpy as np
import pickle
import os
import torch
from scipy.stats import norm

import tslearn.metrics

import itertools
from itertools import combinations

from utils import *
from data_utils import *
from models_solera import *

In [ ]:
data_folder = "data/simulation/spring_simulation/"
suffix = "_static_10_3_3"

n_atoms = 10
n_timesteps = 49
n_in = 4


features_path = os.path.join(data_folder, "all_features"+suffix+".npy")
all_clustering_path = os.path.join(data_folder, "all_clustering"+suffix+".pkl")

features = np.load(features_path)
with open(all_clustering_path, 'rb') as f:
    all_clustering = pickle.load(f)
    
examples = np.zeros((features.shape[0], n_atoms, n_timesteps, n_in)) #feke examples
ground = build_ground_spring(examples) #fake ground

In [ ]:
n_examples = examples.shape[0]
train_idx = int(n_examples*0.6)

valid_idx = int(n_examples*0.8)

examples_train = examples[:train_idx, :, :, :]
features_train = features[:train_idx,:,:]
clusters_train = all_clustering[:train_idx]

examples_valid = examples[train_idx:valid_idx,:,:,:]
features_valid = features[train_idx:valid_idx,:,:]
clusters_valid = all_clustering[train_idx:valid_idx]

examples_test = examples[valid_idx:,:,:,:]
features_test = features[valid_idx:,:,:]
clusters_test = all_clustering[valid_idx:]

In [ ]:
ssvm = SoleraSVM(n_features=6)
ssvm.fit(examples_train, ground ,clusters_train, n_iters=5000, verbose=1, 
         verbose_iters=500, pairwise_features=features_train)

In [ ]:
# Test on Test dataset
recalls = []
precisions = []
F1s = []

for i in range(len(examples_test)):
    example = examples_test[i]
    label = clusters_test[i]
    features = features_test[i]
    predicted, _ = ssvm.predict(example, ground, features)
    recall, precision, F1 = compute_groupMitre(label, predicted)
    recalls.append(recall)
    precisions.append(precision)
    F1s.append(F1)
    
print("Average recall: ", np.mean(recalls))
print("Average precision: ", np.mean(precisions))